In [2]:
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = 'simple_white'
from dataclasses import dataclass

from energy import energy_asymmetric, energy_distribution, energy_M
from continuous import continuous_distribution, continuous_M, continuous_shoot
from torque import torque_distribution, torque_M, torque_shoot, first_point, propagate, boundary
from continuous_root import root_v2


# Model Comparison For Known Values

In [3]:
@np.vectorize
def A1_func(x):
    #defined such that x=0 is the edge of the Ms1
    if x < 2:
        return 3
    elif x < 3:
        return 2
    else:
        return 1
    
@np.vectorize
def Ms1_func(x):
    #defined such that x=0 is the edge of the Ms1
    if x < 2:
        return 1.5
    elif x < 3:
        return 1
    else:
        return 0.5
    
    

@np.vectorize
def A2_func(x):
    #defined such that x=0 is at the interface of Ms2
    if x < 2:
        return 1
    else:
        return 3
    
@np.vectorize
def Ms2_func(x):
    #defined such that x=0 is at the interface of Ms2
    if x < 2:
        return 0.5
    else:
        return 1.5

In [4]:
H = 1

Ms1 = 1.5
Ms2 = 1.5
ht1 = 4
ht2 = 4
d = 2
N1 = int(ht1*10/d)
N2 = int(ht2*10/d)
discrete_x1 = np.linspace(0,ht1,N1)
discrete_x2 = np.linspace(0,ht2,N2)


A1 = A1_func(discrete_x1[:-1])
A2 = A2_func(discrete_x2[:-1])
# A1 = A1_60*np.ones(N1-1)
# A2 = A2_60*np.ones(N2-1)
d_x = np.linspace(0,ht1+ht2,(N1+N2))-4



Ms2 = Ms2_func(discrete_x2)
Ms1 = Ms1_func(discrete_x2)
# Ms1 = Ms1_60*np.ones(N1)
# Ms2 = Ms2_60*np.ones(N2)
a1 = d
a2 = d
J1 = 1.5
J2 = 1.5


t_args = (H, J1, J2, A1, A2, N1, N2, Ms1, Ms2, a1, a2)
e_args = (H, J1, J2, N1, N2, A1, A2, Ms1, Ms2, a1)
# phi0, H, J1, J2, A1, A2, Ms1, Ms2, half_thickness1, half_thickness2
c_args = (H, J1, J2, A1_func, A2_func, Ms1_func, Ms2_func, ht1, ht2)

In [5]:
theta0 = 0.468
print(torque_distribution(theta0, *t_args))
print(torque_shoot(theta0, *t_args, show=True))

[0.468      0.4684511  0.46935371 0.47070863 0.47251706 0.47478063
 0.47750134 0.48068161 0.48432427 0.48843255 0.49301007 0.50034967
 0.50816902 0.51647497 0.52527475 0.534576   0.55368887 0.57332767
 0.59350901 0.61424974 0.72435284 0.70369843 0.6836912  0.66431575
 0.64555694 0.62739987 0.60982993 0.59283281 0.57639448 0.56050123
 0.54513965 0.54053784 0.53645064 0.53287453 0.52980644 0.52724371
 0.52518414 0.52362594 0.52256777 0.5220087 ]
final condition 1.5
thetad  = 0.52200869751212
thetad_1 = 0.5225677654735018
A = 300
a = 2.0
Ms = 0.15000000000000002
-0.018133683534510586


In [6]:
# energy minimization sometimes finds solutions not between 0 and 2pi, have to bound but haven't yet
energy_theta = energy_distribution(*e_args)%(np.pi)
energy_theta = np.where(energy_theta < np.pi/2, energy_theta, np.pi-energy_theta)
print(energy_theta[0])


torque_theta0 = root_v2(torque_shoot, 1e-3, np.pi+1e-3, tol=1e-7, args=t_args, show=True)
torque_theta = torque_distribution(torque_theta0, *t_args)


continuous_theta0 = root_v2(continuous_shoot, 1e-5, np.pi+1e-3, tol=1e-6, step=1e-2, args=c_args)[0]
c_x, continuous_theta, _ = continuous_distribution(continuous_theta0, *c_args)






0.46694620342380944
0.001 (0.001, 0.5795278774825392)
guess=0.5795278774825392, new_min=8.261175321599978e-08, found_min=8.261175321599978e-08, False


In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=d_x, y=np.rad2deg(torque_theta), mode="markers", marker_symbol="diamond", marker_size=9, name="Disc. Torque"))
# fig.add_trace(go.Scatter(y=np.rad2deg(theta2), mode="markers", marker_symbol="square", marker_size=9, name="same start"))
fig.add_trace(go.Scatter(x=d_x, y=np.rad2deg(energy_theta), mode="markers", name="Energy"))
fig.add_trace(go.Scatter(x=c_x-4, y=np.rad2deg(continuous_theta), mode="lines", line=dict(color='red'), name="Cont. Torque"))
fig.add_vline(x=0, opacity=1, line_dash="dash", line_width=2)
fig.add_annotation(text="SL", showarrow=False, x=0, y=0.1, xanchor="left", yref="y domain", font_size=25)

fig.update_layout(xaxis_title="x [nm]", yaxis_title="Angle [deg]", legend_xanchor="right", font_size=22, height=500, width=1200)


fig.show()


In [8]:
field = np.linspace(1e-2,6,80)




t_mag = torque_M(field, *t_args[1:], tol=1e-7, step=1e-3)
e_mag = energy_M(field, *e_args[1:])
c_mag = continuous_M(field, *c_args[1:], tol=1e-7, step=1e-3, min_tol=1e-6)

In [9]:
fig = go.Figure().set_subplots(rows=2, cols=1, row_heights=(0.8,0.2), vertical_spacing=0.05, shared_xaxes=True)
fig.add_trace(go.Scatter(x=field, y=e_mag, mode="lines+markers", line=dict(dash="dot", color='red'), marker_color="red", name="Energy", opacity=0.7))
fig.add_trace(go.Scatter(x=field, y=t_mag, mode="lines", line=dict(color="black"), name="Disc. Torque", opacity=1))

fig.add_trace(go.Scatter(x=field, y=c_mag, mode="lines", line=dict(dash="dash", color="#00BFFF"), name="Cont. Torque", opacity=1))
fig.update_layout(yaxis_title="M", legend_xanchor="right", legend_y=0.5, font_size=23, height=350)
fig.update_xaxes(title="Field [T]", row=2, col=1)
fig.update_yaxes(title="Res.", row=2, col=1, dtick=0.01)
fig.add_trace(go.Scatter(x=field, y=e_mag-t_mag, mode="markers+lines", line_color="black", marker_color="black", showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=field, y=e_mag-c_mag, mode="markers+lines", line=dict(dash="dash", color="#00BFFF"), marker_color="#00BFFF", showlegend=False), row=2, col=1)
fig.update_layout(width=1000, height=600)
fig.show()

# Fitting

In [10]:

@dataclass
class Material:
    Ms: float = None
    A: float = None

@dataclass
class Multilayer:
    symmetry: str = None
    A1: any = None
    A2: any = None
    Ms1: any = None
    Ms2: any = None
    

# example only
Co90Ru10 = Material(Ms=0.964, A=1.41)
Co85Ru15 = Material(Ms=0.768, A=0.5)
Co = Material(Ms=1.389, A=2.84)



x = np.arange(0,4,0.2)



    
A1_60 = Co.A
Ms1_60 = Co.Ms
A2_60 = Co90Ru10.A
Ms2_60 = Co90Ru10.Ms


A1_61 = Co.A
Ms1_61 = Co.Ms

@np.vectorize
def A2_61(x):
    if x < 0.8:
        return Co.A
    else:
        return Co85Ru15.A
    
@np.vectorize
def Ms2_61(x):
    if x < 0.8:
        return Co.Ms
    else:
        return Co85Ru15.Ms   
A2_61_arr = A2_61(x[:-1])
Ms2_61_arr = Ms2_61(x)




# a dictionary to store the multilayers
multilayer_dict = {"60":Multilayer(symmetry="asymmetric", A1=A1_60, A2=A2_60, Ms1=Ms1_60, Ms2=Ms2_60),
                   "61":Multilayer(symmetry="asymmetric", A1=A1_61, A2=A2_61_arr, Ms1=Ms1_61, Ms2=Ms2_61_arr)}

In [11]:
filename = "RAM5-61-295k.dat"
# filename = "RAM5-60-295k.dat"

key = filename.split("-")[1]
try:
    structure = multilayer_dict[key]
except:
    structure = None
    
field, momentRaw = np.genfromtxt(filename, delimiter = '\t', unpack = True, usecols=(3,5))

#resort fields into ascending order
sort = np.argsort(field)
field = field[sort]
momentRaw = momentRaw[sort]

#eliminate small field values for this fit (order matters)
momentRaw = momentRaw[field > 0.2]
field = field[field > 0.2]

_, index = np.unique(field, return_index=True)
field = field[index]

momentRaw = momentRaw[index]




#extract the last N amount of points
length_of_last_points = 200
tail_x = field[-length_of_last_points:] 
tail_y = momentRaw[-length_of_last_points:]
#fit the tail
a, b = np.polyfit(tail_x, tail_y, 1)



#get some information
moment = momentRaw - field*a
avg_moment = np.mean(moment[-length_of_last_points:])

norm_moment = moment/avg_moment
print(f"a = {a:.2e}\nb = {b:.2e}\navg_moment = {avg_moment:.2e}")
# sigma = 0.006*np.ones(len(field))
sigma_2 = np.std(norm_moment[-length_of_last_points:])


field = field[0:-length_of_last_points:25]
norm_moment =  norm_moment[0:-length_of_last_points:25]
sigma = sigma_2 * np.ones(len(field))
print(len(field))


J1 = 3.299
J2 = 1.539




a = -4.91e-05
b = 3.77e-04
avg_moment = 3.77e-04
76


In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=field, y=norm_moment, error_y=dict(array=sigma), mode="markers", name='data'))

#fig.add_annotation(text=f"J1 = {pOpt[0]:.3f}{sc.pm}{pError[0]:.3f}, J2 = {pOpt[1]:.3f}{sc.pm}{pError[1]:.3f}", x=1, y=0, showarrow=False, xref="x domain", yref="y domain")
fig.update_xaxes(title=f"H [T]")
fig.update_yaxes(title="M", range=(0,1.1))

fig.update_layout(legend_x=1, legend_xanchor="right", legend_y=0.65, font_size=25, height=500, width=800)
fig.show()

In [13]:
N1 = 20
N2 = 20
a1 = 2
x = np.arange(0, 2*N1, a1)/10
use_energy = False
if use_energy:
    f = lambda H, J1, J2: energy_M(H, J1, J2, N1, N2, structure.A1, structure.A2, structure.Ms1, structure.Ms2, a1)
    print("you are using the energy model!")
    method = "energy"
    
elif structure.symmetry == "symmetric":
    print("SYMMETRIC SAMPLE")
    f = lambda H,J1,J2: torque_M(H, J1, J2, structure.A1, structure.A2, N1, N2,  structure.Ms1, structure.Ms2, a1, a1, tol=1e-7, step=1e-3, min_tol=1e-5)
    method = "torque"
else:
    print("ASYMMETRIC SAMPLE")
    f = lambda H,J1,J2: torque_M(H, J1, J2, structure.A1, structure.A2, N1, N2, structure.Ms1, structure.Ms2, a1, a1, tol=1e-7, step=1e-3, min_tol=1e-5)
    method = "torque"
print(structure)

ASYMMETRIC SAMPLE
Multilayer(symmetry='asymmetric', A1=2.84, A2=array([2.84, 2.84, 2.84, 2.84, 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ]), Ms1=1.389, Ms2=array([1.389, 1.389, 1.389, 1.389, 0.768, 0.768, 0.768, 0.768, 0.768,
       0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768,
       0.768, 0.768]))


In [14]:
# initial guesses for J1 and J2
J1 = 2.3
J2 = 1.1



pOpt, pCov, info, msg, code = curve_fit(f, field, norm_moment, p0 = [J1,J2], full_output=True, bounds = ((0,0),(4,3)), diff_step=0.08, sigma=sigma, absolute_sigma=True)


pError = np.sqrt(np.diag(pCov))
print(info)
print(pOpt)
print(pError)

{'nfev': 16, 'fvec': array([-4.80045105e+00,  2.98096115e-03,  2.16502047e+00,  2.53537126e+00,
        5.27017428e+00,  3.47296331e+00,  2.99659879e+00,  1.44203347e+00,
        3.97052774e-01,  1.43342296e+00,  3.52457478e-01, -7.46136880e-02,
       -1.00819999e+00, -5.37818026e-01, -1.69586575e+00, -1.09410708e+00,
       -1.47470718e+00, -1.44122757e+00, -1.38583632e+00, -2.10208110e+00,
       -2.13264176e+00, -1.78660640e+00, -3.54654750e+00, -2.72005494e+00,
       -1.57678600e+00, -1.57256069e+00, -2.58438794e+00, -3.20750589e+00,
       -3.67505072e+00, -2.86456602e+00, -2.29371446e+00, -2.03396690e+00,
       -1.50603787e+00, -5.31554467e-01, -2.24453728e+00, -1.86969358e+00,
       -6.68140360e-01, -1.12086068e+00, -3.52001240e-01,  8.55213265e-01,
       -1.71958062e+00,  9.86221749e-01,  3.41535170e-01,  6.52111344e-01,
        1.69915244e+00,  1.68544994e-01,  1.89249735e+00,  7.62285712e-01,
        1.33921470e+00,  3.70905863e+00,  1.11916854e+00,  4.33754520e+00,
    

In [16]:
y_fit = f(field, *pOpt)
norm_res = (norm_moment-y_fit)/(sigma)


In [17]:
fig = go.Figure().set_subplots(rows=2, cols=1, row_heights=(0.6,0.4), shared_xaxes=True, vertical_spacing=0.05)
fig.add_trace(go.Scatter(x=field, y=norm_moment, error_y=dict(array=sigma), mode="markers", name='data'))
fig.add_trace(go.Scatter(x=field, y=y_fit, mode="lines", line_color='red', name='fit'))
fig.add_trace(go.Scatter(x=field, y=norm_res, mode="markers", marker_color='black', showlegend=False), row=2, col=1)
#fig.add_annotation(text=f"J1 = {pOpt[0]:.3f}{sc.pm}{pError[0]:.3f}, J2 = {pOpt[1]:.3f}{sc.pm}{pError[1]:.3f}", x=1, y=0, showarrow=False, xref="x domain", yref="y domain")
fig.update_xaxes(title=f"H [T]", row=2, col=1)
fig.update_yaxes(title="M", range=(0,1.1), row=1, col=1)
fig.update_yaxes(title="Norm. Res.", row=2, col=1, range=(-1.1*max(abs(norm_res)), 1.1*max(abs(norm_res))))
fig.update_layout(legend_x=1, legend_xanchor="right", legend_y=0.65, height=700, width=1200, font_size=25)
fig.show()
